In [1]:
 %run "../Libraries/ADP_Farmatic_Process"

In [2]:
%run "../Libraries/ADP_Farmatic_QA"

In [3]:
process = "FMT_SL"
logger_name = "FMT_Data_ValidateFiles"
level_action = "DEFAULT"
log_level = "DEFAULT"

In [4]:
#################################################################################
""" Main Process for Filename Validation

"""
 #Who                 When           What
 #Victor Salesa       06/11/2018     Initial Version
 #Victor Salesa       20/11/2018     Added QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA 
 #                                   process to serialize file name validation information
 #                                   to Database and FileSystem
 #Victor Salesa       07/02/2019     Replace dbutils.fs.ls function to blob_ls function
 #Victor Salesa       26/02/2019     Added dbutils.notebook.exit 
 #Victor Salesa       04/03/2019     Delete file sampling lines and leave just sellout
 #Victor Salesa       18/03/2019     Take the START_DATE for the really very begining of the validate files process
 #Ana Perez           28/03/2019     Included log managment and exception managment
################################################################################

# Process Landing Files
try: 
  #Vars to control process
  exception_source = logger_name+'.DEFAULT'
  exception_messages={ logger_name+'.DEFAULT':'Undefined Error',
                       logger_name+'.MOVE_FILES':'Fail when moving the files',
                       logger_name+'.WRITE_QA':'Fail when writing to QA please Investigate and Recover Manually Calling ManualRecoverPharmaticFilesQAData'
  }

  #Create parameter to launch task from notebook launcher
  try:
    level_action = dbutils.widgets.get("level_action") 
    log_level = dbutils.widgets.get("log_level")  
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)   
  except Exception as err:
    ADP_log_info(process, logger_name, "DEFAULT", "DEFAULT", "Internal Call parameter was not Created Please Re-run", sys._getframe().f_code.co_name)   

    dbutils.widgets.text("level_action", "DEFAULT","Level Action")
    dbutils.widgets.text("log_level","DEFAULT","Log Level")

    level_action = dbutils.widgets.get("level_action") 
    log_level = dbutils.widgets.get("log_level")
  #end exception 
  
  # Date and Time of the begining to this process
  start_date = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
  
  # Create File list from Pharmatic Landing Folder
  filelist = blob_ls(__PHARMATIC_LANDING_BASE_PATH__)
  
  if len(filelist)!=0:
    
    ######### Restrict Files to be processed to Sellout Files ########
    filelist = [file for file in filelist if any(pattern in file.name for pattern in __PHARMATIC_FILES_INCLUDED_PATTERN__)]
    ##################################################################
    
    if len(filelist)!=0:
      #Add column with Start Process Timestamp
      file_validation_start_df = createDFFromList(filelist).withColumn("START_DATE", to_timestamp(lit(start_date), 'yyyy-MM-dd HH:mm:ss')).repartition(__PARTITIONS_DEFAULT__)
      
      #Validate File Names
      file_validation_files_df = file_validation_start_df.transform(ValidatePharmaticFiles("path","name")) 
      
      #Validate File Names and Move to the proper folder (error/tobeprocessed)
      exception_source = logger_name+'.MOVE_FILES'
      file_validation_move_files_df = file_validation_files_df.transform(DistributePharmaticFiles("path","name",lit(__PHARMATIC_TOBEPROCESSED_BASE_PATH__),lit(__PHARMATIC_ERROR_BASE_PATH__),lit(__PHARMATIC_ARCHIVE_BASE_PATH__)))
      file_validation_move_files_df.cache()
      file_validation_move_files_df.count()
  
      #Transform Data obtained from previous processes and write to QA BLOB/DDB 
      exception_source = logger_name+'.WRITE_QA'
      file_validation_write_QA = file_validation_move_files_df.transform(QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA)
    
    else:
      ADP_log_warning(process, logger_name, level_action, log_level, "END No files found", sys._getframe().f_code.co_name)
    #end if len(filelist)!=0: 
  else:
    ADP_log_warning(process, logger_name, level_action, log_level, "END No files found", sys._getframe().f_code.co_name)
  #end if len(filelist)!=0: 
  
  ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name) 
  
except Exception as err:
  try:
    error_message = exception_messages[exception_source]
  except Exception as e:
    error_message    = "Undefined Error"
    exception_source = logger_name+".DEFAULT"
    
  if exception_source == logger_name+'.MOVE_FILES':
    ManualRecoverPharmaticFilesQAData()
  else: 
    if exception_source == logger_name+'.WRITE_QA':
      print('Pending to define if rollback what was written to QA')
      #       ManualRecoverPharmaticFilesQAData()
    #end if exception_source == logger_name+'.WRITE_QA'
  #end if exception_source == logger_name+'.MOVE_FILES':
  
  ADP_log_exception(process, logger_name, level_action, log_level, error_message, sys._getframe().f_code.co_name,  sys.exc_info())
  raise Exception(err)

In [5]:
%sh cat /dbfs/mnt/blob/log/ADP-Log-SellOut-active.log

2019-05-07 13:01:31 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:01:31 WARN FMT_Data_ValidateFiles: [Main: END No files found]
2019-05-07 13:01:31 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:08:43 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:08:43 WARN FMT_Data_ValidateFiles: [Main: END No files found]
2019-05-07 13:08:43 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:08:52 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:08:52 WARN FMT_Data_ValidateFiles: [Main: END No files found]
2019-05-07 13:08:52 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:09:12 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:09:12 WARN FMT_Data_ValidateFiles: [Main: END No files found]
2019-05-07 13:09:12 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:09:28 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:09:28 WARN FMT_Data_ValidateFiles: [Main: END No files found]
2019-05-07 13:09:28 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:22:08 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:22:08 WARN FMT_Data_ValidateFiles: [Main: END No files found]
2019-05-07 13:22:08 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:24:21 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:24:21 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:24:59 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:24:59 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:32:19 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:32:20 INFO FMT_Data_ValidateFiles: [ValidatePharmaticFiles: BEGIN]
2019-05-07 13:32:20 INFO FMT_Data_ValidateFiles: [ValidatePharmaticFiles: END]
2019-05-07 13:32:20 INFO FMT_Data_ValidateFiles: [inner: BEGIN]
2019-05-07 13:32:20 DEBUG FMT_Data_ValidateFiles: [inner: Start ValidatePharmaticFiles]
2019-05-07 13:32:20 INFO FMT_Data_ValidateFiles: [inner: END: After this line it will prepare and validate files]
2019-05-07 13:32:20 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:33:41 INFO FMT_Data_ValidateFiles: [Main: BEGIN]
2019-05-07 13:33:41 INFO FMT_Data_ValidateFiles: [ValidatePharmaticFiles: BEGIN]
2019-05-07 13:33:41 INFO FMT_Data_ValidateFiles: [ValidatePharmaticFiles: END]
2019-05-07 13:33:41 INFO FMT_Data_ValidateFiles: [inner: BEGIN]
2019-05-07 13:33:41 DEBUG FMT_Data_ValidateFiles: [inner: Start ValidatePharmaticFiles]
2019-05-07 13:33:41 INFO FMT_Data_ValidateFiles: [inner: END: After this line it will prepare and validate files]
2019-05-07 13:33:42 INFO FMT_Data_ValidateFiles: [DistributePharmaticFiles: BEGIN]
2019-05-07 13:33:42 INFO FMT_Data_ValidateFiles: [inner: BEGIN]
2019-05-07 13:33:42 INFO FMT_Data_ValidateFiles: [inner: END: After this line it will prepare and validate files]
2019-05-07 13:33:48 INFO FMT_Data_ValidateFiles: [Main: END]
2019-05-07 13:36:08 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: Before saveAsCanonical ctl_process_file_val: 4 rows ]
2019-05-07 13:36:13 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: After saveAsCanonical ctl_process_file_val]
2019-05-07 13:36:13 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: Before saveToDB CTL.PROCESS_FILE_VAL: 4 rows ]
2019-05-07 13:36:14 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: After saveToDB CTL.PROCESS_FILE_VAL]
2019-05-07 13:38:18 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: Before saveAsCanonical ctl_process_file_val: 4 rows ]
2019-05-07 13:38:23 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: After saveAsCanonical ctl_process_file_val]
2019-05-07 13:38:23 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: Before saveToDB CTL.PROCESS_FILE_VAL: 4 rows ]
2019-05-07 13:38:24 DEBUG FMT_Data_ValidateFiles: [QA_SERIALIZE_FARMATIC_FILE_NAME_VALIDATION_DATA: After saveToDB CTL.PROCESS_FILE_VAL]